In [2]:
import subprocess
import os
import pandas as pd

def ucitaj_sekvencu_iz_fajla(fajl_putanja):
    """
    Učitava sekvencu iz FASTA formata fajla, uklanja zaglavlje i vraća sekvencu kao string.
    """
    with open(fajl_putanja, 'r') as fajl:
        linije = fajl.readlines()
    
    # Preskakanje prvog reda (zaglavljem koji počinje sa ">") i spajanje ostatka u sekvencu
    sekvenca = "".join([linija.strip() for linija in linije[1:]])  # Početni red je zaglavlje
    
    return sekvenca

def parsiraj_iupred3(iupred_izlaz):
    """
    Parsira izlaz iz IUPred3 programa i klasifikuje svaku poziciju u sekvenci kao 'uređenu' ili 'neuređenu'.
    """
    rezultati = []
    for linija in iupred_izlaz.splitlines():
        if linija.strip() and not linija.startswith("#"):
            delovi = linija.split()
            if len(delovi) >= 3:
                pozicija = int(delovi[0])  # Pozicija
                aminokiselina = delovi[1]  # Aminokiselina
                skor = float(delovi[2])  # IUPred skor
                klasifikacija = "uređena" if skor <= 0.5 else "neuređena"
                rezultati.append((pozicija, aminokiselina, skor, klasifikacija))
    
    return rezultati

def generisi_fasta_za_prozor(sekvenca, indeks):
    """
    Generiše FASTA fajl za svaki prozor.
    """
    fasta_putanja = f"temp_prozor_{indeks}.fasta"
    with open(fasta_putanja, 'w') as f:
        f.write(f">prozor_{indeks}\n{sekvenca}")
    return fasta_putanja

def pokreni_iupred3_na_fajlu(sekvenca, indeks):
    """
    Pokreće IUPred3 na datoj sekvenci iz privremenog FASTA fajla i vraća njegov izlaz.
    """
    # Generišemo FASTA fajl za sekvencu
    fasta_putanja = generisi_fasta_za_prozor(sekvenca, indeks)

    # Pokrećemo IUPred3 sa fajlom kao argumentom
    iupred3_putanja = "/home/pc/Desktop/iupred3/iupred3.py"  # Putanja do IUPred3 skripte
    rezultat = subprocess.run(
        ['python3', iupred3_putanja, fasta_putanja, 'long'],  # Pokrećemo IUPred3 sa fajlom
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    # Brisanje privremenog FASTA fajla
    os.remove(fasta_putanja)

    if rezultat.stderr:
        print(f"Greška pri pokretanju IUPred3: {rezultat.stderr}")
        return None

    return rezultat.stdout

def generisi_prozore_i_klasifikuj(sekvenca, min_duzina=30, max_duzina=100):
    podaci = []

    for duzina in range(min_duzina, max_duzina + 1):
        for i in range(len(sekvenca) - duzina + 1):
            podsekvenca = sekvenca[i:i + duzina]
            pozicija = f"{i+1}-{i+duzina}"

            # Pokrećemo IUPred3 za trenutni prozor
            iupred_izlaz = pokreni_iupred3_na_fajlu(podsekvenca, pozicija)

            if iupred_izlaz is None:
                print(f"IUPred3 nije uspeo za prozor {pozicija}. Preskačem...")
                continue

            # Parsiramo rezultate i dobijamo klasifikacije
            iupred_parsirani_rezultati = parsiraj_iupred3(iupred_izlaz)

            # Uzimamo klasifikacije iz IUPred3 rezultata
            podsekvenca_oznake = [result[3] for result in iupred_parsirani_rezultati]
            
            # Klasifikujemo prozor kao većinski 'uređena' ili 'neuređena'
            if podsekvenca_oznake.count("uređena") > len(podsekvenca_oznake) / 2:
                oznaka_prozora = "uređena"
            else:
                oznaka_prozora = "neuređena"
            
            podaci.append((duzina, pozicija, podsekvenca, oznaka_prozora))
    
    return podaci


fajl_putanja = "Nucleoprotein/YP_138520.1 |nucleoprotein.fasta"  # Putanja do FASTA fajla
sekvenca = ucitaj_sekvencu_iz_fajla(fajl_putanja)

podaci = generisi_prozore_i_klasifikuj(sekvenca)

df_sekvence = pd.DataFrame(podaci, columns=["Duzina", "Pozicija", "Sekvenca", "IUPRED3 oznaka"])

csv_putanja = "Nucleoprotein/Klasifikovane_sekvence_IUPRED3_30-100.csv"
df_sekvence.to_csv(csv_putanja, index=False)

print(df_sekvence.head())


   Duzina Pozicija                        Sekvenca IUPRED3 oznaka
0      30     1-30  MDKRVRGSWALGGQSEVDLDYHKILTAGLS        uređena
1      30     2-31  DKRVRGSWALGGQSEVDLDYHKILTAGLSV        uređena
2      30     3-32  KRVRGSWALGGQSEVDLDYHKILTAGLSVQ        uređena
3      30     4-33  RVRGSWALGGQSEVDLDYHKILTAGLSVQQ        uređena
4      30     5-34  VRGSWALGGQSEVDLDYHKILTAGLSVQQG        uređena
